In [1]:
from codemixeddata import *
from customclassifiers import *
from preprocessor import *
from customutils import *

import posfeatures as pos
import lidffeatures as lidf
import chunkfeatures as chunk

from sklearn_crfsuite import scorers, metrics, CRF

import numpy as np
from sklearn.base import BaseEstimator
from sklearn.cross_validation import cross_val_score, train_test_split, cross_val_predict, KFold
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import make_scorer, accuracy_score

from pipeline import CMSTPipeline


In [2]:
lidf_features = FeatureStacker([
    ('bnc_count', lidf.BNCCountsTransformer()),
    ('lex_norm', lidf.LexNormCountsTransformer()),
    ('in_bnc_or_lexnorm', lidf.LexNormBNCSpecialTransformer()),
    ('hindi_dict', lidf.HindiDictionaryCountsTransformer()),
    ('capitalization', lidf.CapitalizationTransformer()),
    ('cngram', lidf.CharacterNgramTransformer()),
    ('affixes', lidf.AffixesTransformer())
])

lidf_model = Pipeline([
    ('features', lidf_features),
    ('classifier', CRF())
])

pos_features = FeatureStacker([
    ('lex', pos.AddLexTransformer()),
    ('lang', pos.AddLangTransformer()),
    ('computed_pos', pos.ComputedPOSTransformer()),
    ('computed_pos_confidence', pos.PoSConfidenceTransformer()),
    ('affixes', pos.AffixesTransformer()),
    ('en_clusters', pos.HWCTransformer())
  ])

pos_model = Pipeline([
    ('features', pos_features),
    ('classifier', CRF())
])
pos_model.set_params(features__affixes__strategy='all_norm', features__computed_pos__strategy='combined')

chunk_features = FeatureStacker([
    ('lex', pos.AddLexTransformer()),
    ('lang', pos.AddLangTransformer()),
    ('computed_pos', pos.ComputedPOSTransformer()),
    ('en_rest_pos_confidence', pos.PoSConfidenceTransformer()),
    ('context', pos.ContextTransformer()),
    ('affixes', pos.AffixesTransformer()),
    ('en_clusters', pos.HWCTransformer()),
    ('pos', chunk.PoSTransformer()),
    ('pos_context', chunk.POSContextTransformer()),
    ('lex__predicted_pos', chunk.LexPoSTransformer()),
])

chunk_model = Pipeline([
        ('features', chunk_features),
        ('classifier', CRF())
])
chunk_model.set_params(features__affixes__strategy='all_raw', features__computed_pos__strategy='only_en')

norm_model = Pipeline([
        ('classifier', NormEstimator(rebuild=True))
    ])

hpos_model = Pipeline([
        ('classifier', HPOSEstimator())
    ])

In [3]:
X, y = LoadDataSetWFeatures(xlabels=['WORD', 'POSITION', 'EPOS', 'EPOSSCORE', 'LANG', 'NORM', 'HPOS', '_HPOS'], ylabel='POS')

In [4]:
model = CMSTPipeline([
        ('LANG', lidf_model),
         #('NORM', norm_model),
         #('HPOS', hpos_model),
         ('POS', pos_model)
    ])

In [8]:
model.final_step[0]

'POS'

In [7]:
pos_acc = cross_val_score(model, X, y, scorers.flat_accuracy, cv=10)

NameError: global name 'SeperateColumn' is not defined

In [6]:
pos_acc.mean()

NameError: name 'pos_acc' is not defined

In [ ]:
pos_acc